## Imports

In [1]:
%load_ext autoreload
%autoreload 2
%aimport -os, -np, -plt, -mpl, -c, -u, -unp, -ufloat, -umath, -sp, -sc, -fit, -pd, -sys

In [2]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.constants as c
import scipy as sc
phc = c.physical_constants
import sys
sys.path.append('/home/excalibur/pCloudDrive/Informatik/Python/')
from self_written_functions import *
from scipy.interpolate import interp1d
sys.path.append('/home/excalibur/Dropbox/Bachelor')
from main_class import *
import efficiency_functions as ef

In [3]:
plt.style.use('~/pCloudDrive/Informatik/Python/matplotlibrc/notebook')

# Einleitung, Ziel

Mit den folgenden Messungen soll die Effizienz des WLS bestimmt werden.

$$\varepsilon_{WLS}(\lambda) = \varepsilon_{abs}(\lambda) \cdot \varepsilon_{QY}$$

$\varepsilon_{abs}(\lambda)$ wird mit Hilfe des Monochromators gemessen und ist je nach Schichtdicke und Qualität anders.
$\varepsilon_{QY}$ hingegen ist für alle Beschichtungen gleich und kann aus den folgenden drei Messungen bestimmt werden:

* 360, noWLS, noF: Diese Messung wird benötigt, um unbekannte Faktoren wie Blende, Raumwinkel und Intensität der Lampe bestimmen zu können.
* 360, WLS, 450: Dies ist die Hauptmessung. Aus ihr wollen wir $\varepsilon_{QY}$ bestimmen.
* 360, WLS, noF: Diese Messung dient als Crosscheck

# Theorie

## Messung 1: 360, noWLS, noF

Für den Aufbau ergibt sich folgende Formel:

$$ I_{PD}(\theta) = \int R_{lamp} \cdot S_{lamp}(\lambda) \cdot \varepsilon_{Blende} \cdot \varepsilon_{360}(\lambda) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \varepsilon_{Geo}(\theta) \cdot \Delta\Omega$$ 

* $I_{PD}(\theta)$: gemessene Stromstärke
* $R_{lamp}$: von der Lampe abgestrahlte Rate an Photonen (alle Wellenlängen)
* $S_{lamp}(\lambda)$: von der Lampe abgestrahltes Spektrum, normiert; multipliziert mit $R_{lamp}$ ergibt es die Rate an Photonen pro Wellenlänge
* $\varepsilon_{Blende}$: berücksichtigt den durch die Blende weggenommenen Teil des Lichts
* $\varepsilon_{360}(\lambda)$: Transmission des 360nm-Filters
* $E_{\gamma}(\lambda)$: Photonenergie, benötigt um von Photonrate auf Leistung zu kommen für Response-Funktion
* $R_{PD}(\lambda)$: Response-Function der Photodiode, Einheiten $\frac{A}{W}$
* $\varepsilon_{Geo}(\theta)$: geometrischer Korrekturfaktor, gibt an welcher Anteil der Intensität unter welchem Raumwinkel zu finden ist (unbekannt), auch bei dieser Messung vorhanden, da Strahl ausgedehnt ist
* $\Delta\Omega$: von der Photodiode abgedeckter Raumwinkel

Da wir $\varepsilon_{Geo}(\theta)$ nicht genau kennen und unsere gesamte Messung verwenden wollen und nicht nur eine Position, integrieren wir über unsere Messung.

$$I_{total,\Delta\Omega} = \int_0^{\frac{\pi}{2}} I_{PD}(\theta) \cdot \sin(\theta) \cdot d\theta \int_0^{2 \pi} d\varphi = \int R_{lamp} \cdot S_{lamp}(\lambda) \cdot \varepsilon_{Blende} \cdot \varepsilon_{360}(\lambda) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \Delta\Omega \cdot \int_0^{2 \pi} \varepsilon_{Geo}(\theta) \cdot \sin(\theta) \cdot d\theta \cdot d\varphi$$

Das Integral über $\varepsilon_{Geo}(\theta)$ kennen wir, es ist der Anteil des Lichts, der das Dia verlässt (also nicht totalreflektiert wird). Bei dieser Messung sind dies 100% (da es keine Reemission gibt).
Wir definieren ihn als $\varepsilon_{\Omega}$.
 
$$I_{total,\Delta\Omega} = \int R_{lamp} \cdot S_{lamp}(\lambda) \cdot \varepsilon_{Blende} \cdot \varepsilon_{360}(\lambda) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \Delta\Omega \cdot \varepsilon_{\Omega}$$

Nun wollen wir nach den unbekannten Größen umstellen:

$$R_{lamp} \cdot \varepsilon_{Blende} \cdot \Delta\Omega = \frac{I_{total,\Delta\Omega}}{\int S_{lamp}(\lambda) \cdot \varepsilon_{360}(\lambda) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \varepsilon_{\Omega}}$$

Diese Größe werden wir in der nächsten Rechnung benötigen.

## Messung 2: 360, WLS, 450

$$ I_{PD}(\theta) = \int R_{lamp} \cdot S_{lamp}(\lambda) \cdot \varepsilon_{Blende} \cdot \varepsilon_{360}(\lambda) \cdot \varepsilon_{abs}(\lambda) \cdot d\lambda \cdot \varepsilon_{QY} \cdot \int S_{WLS}(\lambda) \cdot \varepsilon_{450}(\lambda) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \varepsilon_{Geo}(\theta) \cdot \Delta\Omega$$

Wieder beseitigen wir die Winkelabhängigkeit durch Integration. Diesmal ist $\varepsilon_{\Omega}$ jedoch nicht eins, da wir das re-emittierte Licht messen, ein Teil also Totalreflektion untergeht. Der Anteil ist abhängig von $n_1$ ($n_2 = 1$ Luft), daher schreiben wir $\varepsilon_{\Omega}(n_1)$.

$$I_{total,\Delta\Omega} = \int R_{lamp} \cdot S_{lamp}(\lambda) \cdot \varepsilon_{Blende} \cdot \varepsilon_{360}(\lambda) \cdot \varepsilon_{abs}(\lambda) \cdot d\lambda \cdot \varepsilon_{QY} \cdot \int S_{WLS}(\lambda) \cdot \varepsilon_{450}(\lambda) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \varepsilon_{\Omega}(n_1)$$

$$\Leftrightarrow$$

$$\varepsilon_{QY} = I_{total,\Delta\Omega} \cdot \frac{1}{\Delta\Omega \cdot R_{lamp} \cdot \varepsilon_{Blende}} \cdot \frac{1}{\int S_{lamp}(\lambda) \cdot \varepsilon_{360}(\lambda) \cdot \varepsilon_{abs}(\lambda) \cdot d\lambda \cdot \int S_{WLS}(\lambda) \cdot \varepsilon_{450}(\lambda) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \varepsilon_{\Omega}(n_1)}$$

Hierbei sind alle Größen bekannt, wobei der erste Bruch aus dem Ergebniss von Messung 1 bestimmt werden kann.

## Messung 3: 360, WLS, noF

Diese Messung wird als Crosscheck benutzt, da sie Teile der beiden vorangegangenen Messungen enthält.

$$ I_{PD}(\theta) = \int R_{lamp} \cdot S_{lamp}(\lambda) \cdot \varepsilon_{Blende} \cdot \varepsilon_{360}(\lambda) \cdot (1 - \varepsilon_{abs}(\lambda)) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \varepsilon_{Geo,1}(\theta) \cdot \Delta\Omega$$  

$$ + \int R_{lamp} \cdot S_{lamp}(\lambda) \cdot \varepsilon_{Blende} \cdot \varepsilon_{360}(\lambda) \cdot \varepsilon_{abs}(\lambda) \cdot d\lambda \cdot \varepsilon_{QY} \cdot \int S_{WLS}(\lambda) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \varepsilon_{Geo,2}(\theta) \cdot \Delta\Omega$$

Wiederum wird über den Raum integriert. Nun muss man jedoch zwischen $\varepsilon_{\Omega,1} = 1$ des transmittierten Strahl und $\varepsilon_{\Omega,2}(n_1)$ des re-emittierten Strahls unterscheiden.

$$ I_{PD,\Delta\Omega} = \Delta\Omega \cdot R_{lamp} \cdot \varepsilon_{Blende} \cdot (\int S_{lamp}(\lambda) \cdot \varepsilon_{360}(\lambda) \cdot (1 - \varepsilon_{abs}(\lambda)) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \varepsilon_{\Omega,1}$$  

$$ + \int S_{lamp}(\lambda) \cdot \cdot \varepsilon_{360}(\lambda) \cdot \varepsilon_{abs}(\lambda) \cdot d\lambda \cdot \varepsilon_{QY} \cdot \int S_{WLS}(\lambda) \cdot E_{\gamma}(\lambda) \cdot R_{PD}(\lambda) \cdot d\lambda \cdot \varepsilon_{\Omega,2})$$

Dies Gleichung kann also zur Überprüfung dienen.